In [ ]:
import numpy as np
from math import e
import random

In [ ]:
def check_collision_vertical(chessboard, queen_row, queen_column):
  collision_count = 0

  #start with assuming given chessboard is a valid solution
  valid_solution = True

  for current_row in range(len(chessboard)):

    if (current_row != queen_row):

      if(chessboard[current_row][queen_column] == "q" or chessboard[current_row][queen_column] == "Q"):
        valid_solution = False
        collision_count += 1

  return valid_solution, collision_count

In [ ]:
def check_collision_horizontal(chessboard, queen_row, queen_column):
  collision_count = 0

  #start with assuming given chessboard is a valid solution
  valid_solution = True

  for current_column in range(len(chessboard)):

    if (current_column != queen_column):

      if(chessboard[queen_row][current_column] == "q" or chessboard[queen_row][current_column] == "Q"):
        valid_solution = False
        collision_count += 1

  return valid_solution, collision_count

In [ ]:
def check_collision_diagonal(chessboard, queen_row, queen_column):
  collision_count = 0
  valid_solution = True

  #start at queen's position to check diagonally
  current_row = queen_row
  current_column = queen_column

  #checking top left diagonal
  while(not(current_row == 0 or current_column == 0)):
    current_row -= 1
    current_column -= 1

    if(chessboard[current_row][current_column] == "q" or chessboard[current_row][current_column] == "Q"):
      collision_count += 1
      valid_solution = False

  #after reaching edge, set position back to current queen's position
  current_row = queen_row
  current_column = queen_column

  #checking top right diagonal
  while(not(current_row == 0 or current_column == 7)):
    current_row -= 1
    current_column += 1

    if(chessboard[current_row][current_column] == "q" or chessboard[current_row][current_column] == "Q"):
      valid_solution = False
      collision_count += 1

  #after reaching edge, set position back to current queen's position
  current_row = queen_row
  current_column = queen_column

  #checking bottom left diagonal
  while(not(current_row == 7 or current_column == 0)):
    current_row += 1
    current_column -= 1

    if(chessboard[current_row][current_column] == "q" or chessboard[current_row][current_column] == "Q"):
      valid_solution = False
      collision_count += 1

  #after reaching edge, set position back to current queen's position
  current_row = queen_row
  current_column = queen_column

  #checking bottom right diagonal
  while(not(current_row == 7 or current_column == 7)):
    current_row += 1
    current_column += 1

    if(chessboard[current_row][current_column] == "q" or chessboard[current_row][current_column] == "Q"):
      valid_solution = False
      collision_count += 1

  return valid_solution, collision_count

In [ ]:
def solution_checker(chessboard):
  valid_solution = True
  #count of all collisions on given chessboard
  board_collisions = 0

  for current_row in range(8):
    for current_column in range(len(chessboard[current_row])):
      if(chessboard[current_row][current_column] == "Q" or
         chessboard[current_row][current_column] == "q"):

        #horizontal_check returns whether collision occured and the count of collisions
        no_horizontal_collision_detected, horizontal_collision_count = check_collision_horizontal(chessboard, current_row, current_column)

        #vertical_check returns whether collision occured and the count of collisions
        no_vertical_collision_detected, vertical_collision_count = check_collision_vertical(chessboard, current_row, current_column)

        #diagonal_check returns whether collision occured and the count of collisions
        no_diagonal_collision_detected, diagonal_collision_count = check_collision_diagonal(chessboard, current_row, current_column)

        #if collisions detected anywhere, set accepting condition to reject
        if(not(no_horizontal_collision_detected) or not(no_vertical_collision_detected) or not(no_diagonal_collision_detected)):
          valid_solution = False

        #add up all collisions
        board_collisions += (vertical_collision_count + horizontal_collision_count + diagonal_collision_count)

  return valid_solution, board_collisions

In [ ]:
def generate_chessboard(fixed_queen_row, fixed_queen_column):
  #set up initial chessboard
  chessboard = [
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.']
  ]

  #place fixed queen
  chessboard[fixed_queen_row][fixed_queen_column] = 'Q'

  for row in range(8):
    #row starts at 0 and stops after 7

    if(row != fixed_queen_row):
      #generate a random number between 0 and 7
      column = random.randint(0, 7)

      #place pieces
      chessboard[row][column] = 'q'

  return chessboard

In [ ]:
def simulated_annealing():
  initial_temperature = 100
  t_k = initial_temperature
  solution_found = False
  solution_list = []

  #take input of student number to place fixed queen
  student_number = input("Enter your student number: ")

  #slice input number to get last 2 digits
  student_number = student_number[len(student_number) - 2 : len(student_number)]
  fixed_queen_row = 7 - (int(student_number[1]) % 8)
  fixed_queen_column = int(student_number[0]) % 8

  #this variable will store the best chessboard found
  #for now it is given a random chessboard to start with
  best_chessboard = generate_chessboard(fixed_queen_row, fixed_queen_column)

  #evaluate the best chessboard
  #get whether solution is valid, and collision count
  best_chessboard_validity, best_chessboard_collisions = solution_checker(best_chessboard)

  #exploring chessboard
  exploring_chessboard = best_chessboard

  #evaluate the exploring chessboard
  #get whether solution is valid, and collision count
  exploring_chessboard_validity, exploring_chessboard_collisions = (best_chessboard_validity, best_chessboard_collisions)


  #keep generating chessboards to find better solutions
  k = 1
  while(not(solution_found)):

    #looping chessboard variable
    looping_chessboard = generate_chessboard(fixed_queen_row, fixed_queen_column)
    #evaluate looping chessboard
    looping_chessboard_validity, looping_chessboard_collisions = solution_checker(looping_chessboard)

    fitness_difference = looping_chessboard_collisions - exploring_chessboard_collisions

    #check whether we should accept worse solution to escape local optima
    if(fitness_difference <= 0 or random.uniform(0, 1) < np.min([e**((-fitness_difference)/t_k), 1])):

      #exploratory chessboard = looping chessboard
      exploring_chessboard = looping_chessboard
      #evaluation of exploring chessboard = evaluation of looping chessboard
      exploring_chessboard_validity, exploring_chessboard_collisions = (looping_chessboard_validity, looping_chessboard_collisions)

    #if better solution found, change to best solution
    if(looping_chessboard_collisions < best_chessboard_collisions):

      best_chessboard = looping_chessboard
      best_chessboard_validity, best_chessboard_collisions = (looping_chessboard_validity, looping_chessboard_collisions)

      if(best_chessboard_collisions == 0):
        solution_found = True

        #create text file and put output in it
        output_file = open("output.txt", "a")
        print(f"fixed_queen_column: {fixed_queen_column + 1}, fixed_queen_row: {fixed_queen_row}", file = output_file)

        for row in range(len(looping_chessboard)):
          print(f"{looping_chessboard[row]}", file = output_file)

        output_file.close()

    #change temperature
    t_k = (t_k * (np.log(2)/np.log(k+1)))
    k += 1

  return best_chessboard, solution_list

In [ ]:
a_solution = simulated_annealing()

Enter your student number: 03


<ipython-input-7-269ebb554b33>:43: RuntimeWarning: overflow encountered in scalar divide
  if(fitness_difference <= 0 or random.uniform(0, 1) < np.min([e**((-fitness_difference)/t_k), 1])):
<ipython-input-7-269ebb554b33>:43: RuntimeWarning: divide by zero encountered in scalar divide
  if(fitness_difference <= 0 or random.uniform(0, 1) < np.min([e**((-fitness_difference)/t_k), 1])):


269752
